In [2]:
import pandas as pd
import pickle

In [3]:
from sentence_transformers import SentenceTransformer

/home/azureuser/Ambarish/env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance
from qdrant_client.models import PointStruct

In [5]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [6]:
filename_pickle = "ENMAX-ENVIRONMENT.pkl"
collection_name='ENMAX'

In [7]:
qdrant_client = QdrantClient(host='localhost', port=6333)

In [9]:
qdrant_client.recreate_collection(
    collection_name=collection_name, 
    vectors_config=VectorParams(size=384, distance=Distance.COSINE),
)

True

In [8]:
def get_encodings():
    if (filename_pickle != ""):
        with open(filename_pickle, 'rb') as f: 
            df_embeddings = pickle.load(f)
    embeddings_all = df_embeddings["embeddings"].tolist()
    Lines = df_embeddings["text"].tolist()
    return embeddings_all,Lines

In [9]:
embeddings_all,Lines = get_encodings()

In [10]:
total_len = len(Lines)

In [19]:
qdrant_client.upsert(
    collection_name="ENMAX",
    points=[
        PointStruct(
            id=i,
            vector=embeddings_all[i],
            payload={"text":Lines[i]}
        )
        for i in range(total_len)
    ]
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [11]:
user_input ="What is Enmax doing with Nested Birds?"

In [12]:
def get_model():    
    model = SentenceTransformer('all-MiniLM-L6-v2')
    return model

In [13]:
q_new = user_input
q_new_embeddings = get_model().encode(q_new)

In [14]:
search_result = qdrant_client.search(
            collection_name=collection_name,
            query_vector=q_new_embeddings,
            query_filter=None,  # We don't want any filters for now
            top=5  # 5 the most closest results is enough
        )

In [15]:
search_result

[ScoredPoint(id=25, version=0, score=0.6282439, payload={'text': 'program with the herd will shift from initial clearing to\n basic maintenance, saving time, money, and reducing\n environmental impact compared to traditional\n herbicide treatments.\n ¹ A significant spill is defined as more than 500 litres, in alignment with industry standards (including Electricity Canada).\n Land and\n biodiversity\n STORY\n ENMAX 2021\n ENVIRONMENTAL, SOCIAL\n & GOVERNANCE REPORT CEO LET TER ESG SCORECARD ABOUT ENMAX SOCIAL GOVERNANCE TCFD APPENDICES\n 29\n ENVIRONMENT\n Protecting biodiversity\n Birds often use power poles or substation equipment\n for perching, roosting and nesting. Birds may also nest\n in areas where project or maintenance work will be\n carried out. Across our operations, ENMAX works to\n protect birds while helping to ensure the reliability of\n our services by identifying high-risk areas, conducting\n nest sweeps prior to work, and delaying work to\n accommodate nesting perio

In [17]:
payloads = [hit.payload for hit in search_result]

In [18]:
payloads

[{'text': 'program with the herd will shift from initial clearing to\n basic maintenance, saving time, money, and reducing\n environmental impact compared to traditional\n herbicide treatments.\n ¹ A significant spill is defined as more than 500 litres, in alignment with industry standards (including Electricity Canada).\n Land and\n biodiversity\n STORY\n ENMAX 2021\n ENVIRONMENTAL, SOCIAL\n & GOVERNANCE REPORT CEO LET TER ESG SCORECARD ABOUT ENMAX SOCIAL GOVERNANCE TCFD APPENDICES\n 29\n ENVIRONMENT\n Protecting biodiversity\n Birds often use power poles or substation equipment\n for perching, roosting and nesting. Birds may also nest\n in areas where project or maintenance work will be\n carried out. Across our operations, ENMAX works to\n protect birds while helping to ensure the reliability of\n our services by identifying high-risk areas, conducting\n nest sweeps prior to work, and delaying work to\n accommodate nesting periods.\n PREVENTING DISTURBANCE TO NESTING BIRDS\n To avoi